In [4]:
import secrets
import hashlib

In [5]:
def random_binary_generator(n):
    # Gets integer n,
    # Generates n random bits and returns it 
    # (Without the 0b)
    random_bits = secrets.randbits(n)
    binary_string = bin(random_bits)[2:]
    binary_string = binary_string.zfill(n)

    return binary_string

In [6]:
def count_one_elements(binary_number):
    # Gets a binary number,
    # Counts '1' elements in binary number and returns it
    count = 0
    for digit in binary_number:
        if digit == '1':
            count += 1
    return count

def rotate_left(binary_number, shift_amount):
    # Gets a binary number and shitf amount,
    # Returns rotated number in binary
    rotated_binary = binary_number[shift_amount:] + binary_number[:shift_amount]
    return rotated_binary

def rotate_right(binary_number, shift_amount):
    # Gets a binary number and shitf amount,
    # Returns rotated number in binary
    rotated_binary = binary_number[-shift_amount:] + binary_number[:-shift_amount]
    return rotated_binary

In [7]:
def binary_xor(*args):
    # Gets anu amount of binary numbers, 
    # (converts them to same lenth if they are not)
    # Calculates XOR of them and then returns it

    max_length = max(len(binary_string) for binary_string in args)
    padded_args = [binary_string.zfill(max_length) for binary_string in args]
    xor_binary_string = ''.join(str(sum(int(bit) for bit in bits) % 2) for bits in zip(*padded_args))

    return xor_binary_string

In [8]:
def hash_binary(binary_string):
    # Gets a binary number,
    # returns hash of it in hex
    decimal_number = int(binary_string, 2)
    byte_data = decimal_number.to_bytes((decimal_number.bit_length() + 7) // 8, byteorder='big')
    sha1_hash = hashlib.sha1(byte_data).hexdigest()

    sha1_hash = '0x' + sha1_hash
    return sha1_hash

## IDs of nodes

In [9]:
# Should be shared
IDNt_bin = '0010101111000011001000000010100100110011000011001010000101010011001010100101110001011000101100111110110010001111100011000111101111111111101100101011110001110010'
IDNr_bin = '0010110111001000001111000010111111100010010100100111101001101110100101011000100001010100000110000001011010010101001010110110101010110000011011110000111001110000'
IDNs_bin = '1010001111000111111001111100111111001000111110101010001011000001110111000101011110101110100001011000111001000000001101101101111011110001010001010101000111000001'
BalN_bin = '1000110111111000011000100100101111111011001101100010000110111101101011110000110001001001001011011101100000010000001001101111111000010110111100000001101010011011'
BS_bin = '1001101011100011111010110101101111000000010110111111101011111100110100001001100101010111010101010010110001010010010111100110101101100000011000011001111110000000'

In [10]:
# Should be sent from Reader => Tag
# Msg 1
PR_bin = '0110001110101001111111111100001110100000010011011010110001100011001011001010101100111011101100000111001101111000101010000011111000111000001100010000100110011000'
CHR_hex = '0x34873dbe9387e9d312bc5e6940bad2dafd175b5c'
TIR_bin = '01010111101111001011101101010011'

## Msg 2

In [11]:
TIT_bin = random_binary_generator(32)

In [12]:
# Extract R0
# IDNt_bin ^ TIR_bin
shift_amount_bin = binary_xor(IDNt_bin, TIR_bin)
shifted_number_bin = rotate_right(PR_bin, count_one_elements(shift_amount_bin))

# shifted_number_bin ^ IDNt_bin ^ TIR_bin
R0_bin = binary_xor(shifted_number_bin, IDNt_bin, TIR_bin)

In [13]:
# CHT
# R0_bin ^ IDNt_bin ^ BalN_bin
pre_CHT_bin = binary_xor(R0_bin, IDNt_bin, BalN_bin)
CHT_hex = hash_binary(pre_CHT_bin)

In [14]:
# PT
# R0_bin ^ IDNs_bin ^ TIT_bin
PT_1_bin = binary_xor(R0_bin, IDNs_bin, TIT_bin)

# TIT_bin ^ IDNt_bin
PT_2_bin = binary_xor(TIT_bin, IDNt_bin)

PT_bin = rotate_left(PT_1_bin ,count_one_elements(PT_2_bin))

In [15]:
# AthR
# CHT_hex ^ R0_bin ^ PT_bin ^ IDNt_bin ^ TIT
pre_AthR = binary_xor(bin(int(CHT_hex, 16))[2:], R0_bin, PT_bin, IDNt_bin, TIT_bin)
AthR_hex = hash_binary(pre_AthR)

In [16]:
msg2 = {
    "CHT_hex": CHT_hex,
    "AthR_hex": AthR_hex,
    "PT_bin": PT_bin,
    "TIT_bin": TIT_bin
}

print(f"CHT_hex = '{CHT_hex}'")
print(f"AthR_hex = '{AthR_hex}'")
print(f"PT_bin = '{PT_bin}'")
print(f"TIT_bin = '{TIT_bin}'")

CHT_hex = '0x62a263ee93426984701a50427ad9f99b615b7eff'
AthR_hex = '0x20bc909d7021476867db51928b99fa4b81301574'
PT_bin = '0010101010001110010010110110001001110100011000001010010111110111011111101000010000101010010011100100100100101010001100000001100010110110001111011010010010111100'
TIT_bin = '01101011000101100010101010001011'
